## Function that returns shifted alphabet

In [1]:
import string

def alphabet(shift):
    return string.ascii_uppercase[shift:] + string.ascii_uppercase[:shift]

In [2]:
alphabets = [alphabet(shift) for shift in range(len(string.ascii_uppercase))]

## Function for encrypting plaintext using autokey cipher

In [3]:
def encipher(plaintext, key):
    ciphertext = ''
        
    for i in range(len(plaintext)):
        if i >= len(key):
            key += plaintext[i - len(key)]

        shift = ord(key[i]) - ord('A')
        letter = ord(plaintext[i]) - ord('A')

        ciphertext += alphabets[shift][letter]

    return ciphertext

## Guess the keylength
We know that the keylength is at most 6 in length.  
We start with 6 and decrement it if our plaintext don't make sense.

In [4]:
keylength = 6

## Clean up the ciphertext
We remove spaces.  
As spaces were removed before encrypting and then reinserted, this should have no effect on the analysis.

In [5]:
ciphertext = '''FRRUU OIIYE AMIRN QLQVR BOKGK NSNQQ IUTTY
IIYEA WIJTG LVILA ZWZKT ZCJQH IFNYI WQZXH
RWZQW OHUTI KWNNQ YDLKA EOTUV XELMT SOSIX
JSKPR BUXTI TBUXV BLNSX FJKNC HBLUK PDGUI
IYEAM OJCXW FMJVM MAXYT XFLOL RRLAA JZAXT
YYWFY NBIVH VYQIO SLPXH ZGYLH WGFSX LPSND
UKVTR XPKSS VKOWM QKVCR TUUPR WQMWY XTYLQ
XYYTR TJJGO OLMXV CPPSL KBSEI PMEGC RWZRI
YDBGE BTMFP ZXVMF MGPVO OKZXX IGGFE SIBRX
SEWTY OOOKS PKYFC ZIEYF DAXKG ARBIW KFWUA
SLGLF NMIVH VVPTY IJNSX FJKNC HBLUK PDGUI
IYEAM HVFDY CULJS EHHMX LRXBN OLVMR'''

ciphertext = ''.join(ciphertext.split())

## Import English word list
Filter out words that are not long enough to enchiper itself.  
Meaning words that are longer than the key length.  
We also want at least 3 letters enciphered to reduce false-positives.

In [6]:
from nltk.corpus import words

at_least_match = 3

long_enough = map(lambda w: w.upper(), filter(lambda w: len(w) >= keylength + at_least_match, words.words()))

## Encipher the remaining words
We now got stubs of text that would occur in the ciphertext, if a word from the list were enciphered with the given keylength.  
We also save the length of the smallest and largest stub. This is so we have a bound on the stubs we care about in the ciphertext.

In [7]:
stubs = {}

min_stub = len(ciphertext)
max_stub = 0

for word in long_enough:
    key = word[:keylength]
    plaintext = word[keylength:]
    
    stub = encipher(plaintext, key)
    
    stub_len = len(stub)
    
    if stub_len > max_stub:
        max_stub = stub_len
        
    if stub_len < min_stub:
        min_stub = stub_len
    
    stubs[stub] = word

## Generate a list of stubs from the ciphertext
For each stub we save were it occured. This is so we can go back and try to decrypt the ciphertext from that position.  
We can also specify a lower cut. This is just to speed up finding the key as larger words will reveal more of the plaintext.  
We might have to lower this limit if we don't find a matching key.

In [8]:
stubs_in_cipher = []
stub_start = []

lower_cut = min_stub + 2

for end in range(lower_cut, max_stub):
    for start in range(len(ciphertext)):
        stubs_in_cipher.append(ciphertext[start:start+end])
        stub_start.append(start)

## Find the intersection of the stubs from the English word list and ciphertext
If we get a match, it means that the word was probably in the plaintext.  

In [9]:
intersection = []

for stub in stubs_in_cipher:
    if stub in stubs:
        word = stubs[stub]
        intersection.append((word, stub_start[stubs_in_cipher.index(stub)]))
        
intersection = set(intersection)

print("Words probably in plaintext:")
[word for word, _ in intersection]

Words probably in plaintext:


['FACETIOUSLY',
 'CRYPTOGRAPH',
 'CRYPTOGRAPHY',
 'TECHNOLOGUE',
 'APPROPRIATE',
 'SEPHARDIC',
 'CRYPTOGRAPHIC']

## Function for deciphering ciphertext using autokey cipher

In [10]:
def decipher(ciphertext, key):
    plaintext = ''
    estimated_keylength = len(key)
        
    for i in range(len(ciphertext)):
        if i >= len(key):
            key += plaintext[i - estimated_keylength]

        shift = -abs(ord(key[i]) - ord('A')) % 26
        letter = ord(ciphertext[i]) - ord('A')

        plaintext += alphabets[shift][letter]

    return plaintext

## Function for deciphering backwards to resolve the key used

In [11]:
def resolve_key(ciphertext, key):
    return decipher(ciphertext[::-1], key[::-1])[::-1][:len(key)]

## Move backwards and resolve the key used
If the word is in fact in the plaintext, we can move backwards deciphering one keylength at the time until we get the key used.  
We count the number of times a word gives us the same key. The key with the highest count is the most probable.  
We can then inspect the whole plaintext we get from using the key found.

In [12]:
keys = {}

for word, start in intersection:
    key = word[:keylength]
    ciphertext_offset = ciphertext[:start]

    start_key = resolve_key(ciphertext_offset, key)
    possible_key = start_key + 'A'*(keylength-len(start_key))

    if possible_key not in keys:
        keys[possible_key] = 0
    
    keys[possible_key] += 1
    
most_probable_key = max(keys, key=keys.get)
plaintext = decipher(ciphertext, most_probable_key)

print('Most probable key:', most_probable_key, 'with', keys[most_probable_key], 'occurences')
print()
print(plaintext)

Most probable key: DATFBA with 4 occurences

CRYPTOGRAPHYCANBESTRONGORWEAKCRYPTOGRAPHICSTRENGTHISMEASUREDINTHETIMEANDRESOURCESITWOULDREQUIRETORECOVERTHEPLAINTEXTTHERESULTOFSTRONGCRYPTOGRAPHYISCIPHERTEXTTHATISVERYDIFFICULTTODECIPHERWITHOUTPOSSESSIONOFTHEAPPROPRIATEDECODINGTOOLHOWDIFFICULTGIVENALLOFTODAYSCOMPUTINGPOWERANDAVAILABLETIMEEVENABILLIONCOMPUTERSDOINGABILLIONCHECKSASECONDITISNOTPOSSIBLETODECIPHERTHERESULTOFSTRONGCRYPTOGRAPHYBEFORETHEENDOFTHEUNIVERSE


## Inspect all the keys found

In [13]:
for key in keys:    
    plaintext = decipher(ciphertext, key)
    
    print('With key:', key)
    print()
    print(plaintext)
    print('\n')

With key: YGQSXL

HLBCXDBXXCDJHUQOIHOXLACZWQHNORMEMGKRWUSUMRNZORJRYBLFQTVYREAONHWUIIDSBNJOWYVBYGXKPVPHTOOQVTLAFEAETLHPSKZXQUAAQULAXTSZQUACJMXYXDAYQEKYLWULTIJMONLSDCVPMECKOGAIYNKNXXNBBEUONZIVGJGZQBZPHCSUIGROQUKFYJRFWTNYFBJZKNKREEKXLCNTFNHQIRJJFACETIOUSLYOCSENZFWTMKZTXYHZKNRQENNILZLFYCQTTDRKONJOFPDVPPWRBGEXJYYRRPWOIYEZSWRZTJOKOFZZNHJNFXGRFBJNMYFXWPNKZBJONNLFRDOVLFOTGFHGSSZIFCDPWNKRVTNAIGKQXNUBRVXXVCPZLLDCLNWKCBNPYBHRRSJLQUAFSCYRVHZ


With key: DATFBA

CRYPTOGRAPHYCANBESTRONGORWEAKCRYPTOGRAPHICSTRENGTHISMEASUREDINTHETIMEANDRESOURCESITWOULDREQUIRETORECOVERTHEPLAINTEXTTHERESULTOFSTRONGCRYPTOGRAPHYISCIPHERTEXTTHATISVERYDIFFICULTTODECIPHERWITHOUTPOSSESSIONOFTHEAPPROPRIATEDECODINGTOOLHOWDIFFICULTGIVENALLOFTODAYSCOMPUTINGPOWERANDAVAILABLETIMEEVENABILLIONCOMPUTERSDOINGABILLIONCHECKSASECONDITISNOTPOSSIBLETODECIPHERTHERESULTOFSTRONGCRYPTOGRAPHYBEFORETHEENDOFTHEUNIVERSE


With key: RJQYMS

OIBWIWUAXISQORQITAHALGRGDNHHZKFHMMZYDRSOXKGCOXYYFYLZBMOBRKPVUEWOTBWVBTYVDVVVJZQNPBEOALOKGMEDFKPLAIHJDDSAQAPHXR